In [22]:
import numpy as np
import tensorflow as tf
import os
import sys
from sklearn.preprocessing import StandardScaler 

In [23]:
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import load_embeddings_and_ids, concatenate_featmats
from Networks import CuratorNet_Precomputation

In [24]:
resnet50 = load_embeddings_and_ids('../data/resnet_embeddings.npy')

In [25]:
embedding_list = [
    resnet50,
]

In [26]:
artwork_ids_set = set()
for embedding in embedding_list:
    if len(artwork_ids_set) == 0:        
        artwork_ids_set.update(embedding['index2id'].values())
    else:
        artwork_ids_set.intersection_update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [27]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [28]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [29]:
concat_featmat.shape

(13297, 2048)

In [30]:
!dir C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m

 O volume na unidade C nÆo tem nome.
 O N£mero de S‚rie do Volume ‚ 789E-3B0A

 Pasta de C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m

08/02/2022  22:02    <DIR>          .
08/02/2022  22:02    <DIR>          ..
08/02/2022  22:02         8.649.608 .data-00000-of-00001
08/02/2022  22:02             1.204 .index
08/02/2022  22:02           239.676 .meta
08/02/2022  22:02               191 checkpoint
08/02/2022  22:02             1.634 train_logs.csv
               5 arquivo(s)      8.892.313 bytes
               2 pasta(s)   151.067.615.232 bytes dispon¡veis


In [31]:
MODEL_PATH = "C:\\Users\\victor_accete\\CuratorNet\\experiments\\curatornet_10m\\"
MODEL_PATH

'C:\\Users\\victor_accete\\CuratorNet\\experiments\\curatornet_10m\\'

In [32]:
print(tf.train.latest_checkpoint(MODEL_PATH))

C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m\


In [33]:
with tf.Graph().as_default():
    network = CuratorNet_Precomputation(
        pretrained_embedding_dim=concat_featmat.shape[1],
        item_layer_units=[200,200],
    )
    gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=0.99,
            allow_growth=True
        )
    config = tf.ConfigProto(gpu_options=gpu_options)
    with tf.Session(config=config) as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_vectors = network.precompute_tensors(sess, concat_featmat)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m\


In [34]:
item_vectors.shape

(13297, 200)

In [35]:
item_vectors.dump(MODEL_PATH + "item_vectors.npy")
with open(MODEL_PATH + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%s\n' % _id)